In [2]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from langdetect import detect
import spacy


In [3]:
def extraire_mots_cles_fr(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en français.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("fr_core_news_sm")
    requete_utilisateur = requete.lower()
    doc = nlp(requete_utilisateur)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT","obj"]) or \
           (token.pos_ == "ADV" ) or \
           (token.pos_ == "PRON" and token.dep_ != "dep") or \
           (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or \
           (token.pos_ == "ADJ" ) or \
           (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]):
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format


In [4]:
def extraire_mots_cles_en(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en anglais.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(requete)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" ) or  (token.dep_ == "punct") or \
           (token.pos_ == "ADV") or (token.dep_ == "pobj") or \
           (token.pos_ == "PROPN" and token.dep_ != "dep") or \
           (token.pos_ == "ADJ" ) :
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format


In [5]:
def detect_language(text):
    """
    Fonction pour détecter la langue d'un texte donné.
    
    Arguments :
        - text : Le texte à analyser
        
    Returns :
        - 'français' si la langue détectée est le français, sinon 'anglais'
    """
    lang = detect(text)
    if lang == 'fr':
        return 'français'
    else:
        return 'anglais'

In [6]:
def format_mot(motcle):
    """
    Fonction pour formater un mot-clé pour une utilisation dans une URL de recherche.
    
    Arguments :
        - motcle : Le mot-clé à formater
        
    Returns :
        - format_motcle : Le mot-clé formaté pour une URL de recherche
    """
    if ' ' in motcle:
        format_motcle = motcle.replace(' ', '+')
    else:
        format_motcle = motcle
    return format_motcle

In [7]:
def main():
    # Demander à l'utilisateur le sujet de recherche
    requete_utilisateur = str(input('Veuillez saisir votre requête : '))

    # Détection de la langue de la requête
    langue = detect_language(requete_utilisateur)

    # Sélection de la fonction d'extraction des mots-clés en fonction de la langue détectée
    if langue == 'français':
        mots_cles_format = extraire_mots_cles_fr(requete_utilisateur)
        print("Langue détectée ==> Français")
    else:
        mots_cles_format = extraire_mots_cles_en(requete_utilisateur)
        print("Langue détectée ==> Anglais")

    # Formatage des mots-clés pour une utilisation dans une URL de recherche
    search_query = format_mot(mots_cles_format)

    # Demander à l'utilisateur le nombre de pages à parcourir
    num_pages = int(input("Entrez le nombre de pages à parcourir : "))

    # Créer le nom du dossier de destination
    folder_name = f"{search_query.replace('+', '-')}-{num_pages}"

    # Créer le dossier s'il n'existe pas
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Initialiser le navigateur Chrome en mode sans tête
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    # Parcourir les pages demandées
    for page_num in range(1, num_pages+1):
        print(f"\nPage {page_num}")
        # Ouvrir la page de recherche de Semantic Scholar
        driver.get(f"https://www.semanticscholar.org/search?q={search_query}&sort=relevance&page={page_num}")

        # Attendre quelques secondes pour permettre au contenu de charger
        time.sleep(5)

        # Obtenir le contenu de la page
        html_content = driver.page_source

        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Trouver tous les liens vers les PDF
        pdf_links = soup.select('a[href$=".pdf"]')

        # Télécharger les PDF et les enregistrer dans le dossier "articles"
        for link in pdf_links:
            pdf_url = link["href"]
            print("Téléchargement du PDF:", pdf_url)
            try:
                response = requests.get(pdf_url)
                # Générer un nom de fichier unique pour chaque PDF
                file_name = f"article_{time.time()}.pdf"
                # Enregistrer le PDF dans le dossier "articles"
                with open(os.path.join(folder_name, file_name), "wb") as f:
                    f.write(response.content)
                print(f"Enregistré sous: {folder_name}/{file_name}")
            except requests.exceptions.RequestException as e:
                print(f"Erreur lors du téléchargement du PDF {pdf_url}: {str(e)}")

    # Fermer le navigateur
    driver.quit()


In [8]:
if __name__ == "__main__":
    main()

Veuillez saisir votre requête : I'm looking for articles on NLP
Langue détectée ==> Anglais
Entrez le nombre de pages à parcourir : 4

Page 1
Téléchargement du PDF: https://arxiv.org/pdf/1902.00751.pdf
Enregistré sous: NLP-4/article_1713797715.27706.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/P19-1355.pdf
Enregistré sous: NLP-4/article_1713797715.766979.pdf
Téléchargement du PDF: https://arxiv.org/pdf/2005.11401.pdf
Enregistré sous: NLP-4/article_1713797715.93277.pdf
Téléchargement du PDF: https://www.aclanthology.org/2022.emnlp-main.340.pdf
Enregistré sous: NLP-4/article_1713797717.1812868.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2020.acl-main.442.pdf
Enregistré sous: NLP-4/article_1713797717.746401.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2021.naacl-main.168.pdf
Enregistré sous: NLP-4/article_1713797718.235768.pdf
Téléchargement du PDF: https://www.aclanthology.org/2021.findings-acl.84.pdf
Enregistré sous: NLP-4/article_1713797719

# 2 Version

In [12]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import spacy
from langdetect import detect
import uuid  # Importer le module uuid pour la génération de noms de fichiers uniques

def extraire_mots_cles_fr(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en français.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("fr_core_news_sm")
    requete_utilisateur = requete.lower()
    doc = nlp(requete_utilisateur)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT","obj"]) or \
           (token.pos_ == "ADV" ) or \
           (token.pos_ == "PRON" and token.dep_ != "dep") or \
           (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or \
           (token.pos_ == "ADJ" ) or \
           (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]):
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def extraire_mots_cles_en(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en anglais.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(requete)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" ) or  (token.dep_ == "punct") or \
           (token.pos_ == "ADV") or (token.dep_ == "pobj") or \
           (token.pos_ == "PROPN" and token.dep_ != "dep") or \
           (token.pos_ == "ADJ" ) :
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def detect_language(text):
    """
    Fonction pour détecter la langue d'un texte donné.
    
    Arguments :
        - text : Le texte à analyser
        
    Returns :
        - 'français' si la langue détectée est le français, sinon 'anglais'
    """
    lang = detect(text)
    if lang == 'fr':
        return 'français'
    else:
        return 'anglais'

def format_mot(motcle):
    """
    Fonction pour formater un mot-clé pour une utilisation dans une URL de recherche.
    
    Arguments :
        - motcle : Le mot-clé à formater
        
    Returns :
        - format_motcle : Le mot-clé formaté pour une URL de recherche
    """
    if ' ' in motcle:
        format_motcle = motcle.replace(' ', '+')
    else:
        format_motcle = motcle
    return format_motcle




def main():
    """
    Fonction principale du programme.
    """
    # Demander à l'utilisateur le sujet de recherche
    requete_utilisateur = input('Veuillez saisir votre requête : ')

    # Détection de la langue de la requête
    langue = detect_language(requete_utilisateur)

    # Sélection de la fonction d'extraction des mots-clés en fonction de la langue détectée
    if langue == 'français':
        mots_cles_format = extraire_mots_cles_fr(requete_utilisateur)
        print("Langue détectée ==> Français")
    else:
        mots_cles_format = extraire_mots_cles_en(requete_utilisateur)
        print("Langue détectée ==> Anglais")

    # Formatage des mots-clés pour une utilisation dans une URL de recherche
    search_query = format_mot(mots_cles_format)

    # Demander à l'utilisateur le nombre de pages à parcourir
    num_pages = int(input("Entrez le nombre de pages à parcourir : "))

    # Créer le nom du dossier de destination
    folder_name = f"{search_query.replace('+', '-')}-{num_pages}"

    # Créer le dossier s'il n'existe pas
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Initialiser le navigateur Chrome en mode sans tête
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    # Parcourir les pages demandées
    for page_num in range(1, num_pages+1):
        print(f"\nPage {page_num}")
        # Ouvrir la page de recherche de Semantic Scholar
        driver.get(f"https://www.semanticscholar.org/search?q={search_query}&sort=relevance&page={page_num}")

        # Attendre quelques secondes pour permettre au contenu de charger
        time.sleep(20)

        # Obtenir le contenu de la page
        html_content = driver.page_source

        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Trouver tous les liens vers les PDF
        pdf_links = soup.select('a[href$=".pdf"]')

        # Télécharger les PDF et les enregistrer dans le dossier "articles"
        for link in pdf_links:
            pdf_url = link["href"]
            print("Téléchargement du PDF:", pdf_url)
            try:
                # Télécharger le PDF depuis le lien
                response = requests.get(pdf_url, stream=True)
                response.raise_for_status()  # Vérifier si la requête a réussi
                # Générer un nom de fichier unique pour chaque PDF
                file_name = f"article_{uuid.uuid4()}.pdf"  # Utiliser uuid.uuid4() pour générer un nom de fichier unique
                # Enregistrer le PDF dans le dossier de destination
                with open(os.path.join(folder_name, file_name), "wb") as f:
                    f.write(response.content)
                print(f"Enregistré sous: {folder_name}/{file_name}")
            except requests.exceptions.RequestException as e:
                print(f"Erreur lors du téléchargement du PDF {pdf_url}: {str(e)}")
            except Exception as ex:
                print(f"Une erreur inattendue s'est produite : {str(ex)}")

    # Fermer le navigateur
    driver.quit()

if __name__ == "__main__":
    main()

Veuillez saisir votre requête : I'm looking for articles on NLP
Langue détectée ==> Anglais
Entrez le nombre de pages à parcourir : 4

Page 1
Téléchargement du PDF: https://arxiv.org/pdf/1902.00751.pdf
Enregistré sous: NLP-4/article_989f9a38-e214-4208-933b-381bef175765.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/P19-1355.pdf
Erreur lors du téléchargement du PDF https://www.aclweb.org/anthology/P19-1355.pdf: 406 Client Error: Not Acceptable for url: https://www.aclweb.org/anthology/P19-1355.pdf
Téléchargement du PDF: https://arxiv.org/pdf/2005.11401.pdf
Enregistré sous: NLP-4/article_6571d02d-4d56-4eba-9bd1-ccbaa3398bf9.pdf
Téléchargement du PDF: https://www.aclanthology.org/2022.emnlp-main.340.pdf
Enregistré sous: NLP-4/article_d012e96f-22e9-4550-a13e-335bba7aeb51.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2020.acl-main.442.pdf
Erreur lors du téléchargement du PDF https://www.aclweb.org/anthology/2020.acl-main.442.pdf: 406 Client Error: Not Acceptable f

# 3 Version

In [13]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import spacy
from langdetect import detect
import uuid  # Importer le module uuid pour la génération de noms de fichiers uniques

def extraire_mots_cles_fr(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en français.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("fr_core_news_sm")
    requete_utilisateur = requete.lower()
    doc = nlp(requete_utilisateur)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT","obj"]) or \
           (token.pos_ == "ADV" ) or \
           (token.pos_ == "PRON" and token.dep_ != "dep") or \
           (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or \
           (token.pos_ == "ADJ" ) or \
           (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]):
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def extraire_mots_cles_en(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en anglais.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(requete)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" ) or  (token.dep_ == "punct") or \
           (token.pos_ == "ADV") or (token.dep_ == "pobj") or \
           (token.pos_ == "PROPN" and token.dep_ != "dep") or \
           (token.pos_ == "ADJ" ) :
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def detect_language(text):
    """
    Fonction pour détecter la langue d'un texte donné.
    
    Arguments :
        - text : Le texte à analyser
        
    Returns :
        - 'français' si la langue détectée est le français, sinon 'anglais'
    """
    lang = detect(text)
    if lang == 'fr':
        return 'français'
    else:
        return 'anglais'

def format_mot(motcle):
    """
    Fonction pour formater un mot-clé pour une utilisation dans une URL de recherche.
    
    Arguments :
        - motcle : Le mot-clé à formater
        
    Returns :
        - format_motcle : Le mot-clé formaté pour une URL de recherche
    """
    if ' ' in motcle:
        format_motcle = motcle.replace(' ', '+')
    else:
        format_motcle = motcle
    return format_motcle


def main():
    """
    Fonction principale du programme.
    """
    # Demander à l'utilisateur le sujet de recherche
    requete_utilisateur = input('Veuillez saisir votre requête : ')

    # Détection de la langue de la requête
    langue = detect_language(requete_utilisateur)

    # Sélection de la fonction d'extraction des mots-clés en fonction de la langue détectée
    if langue == 'français':
        mots_cles_format = extraire_mots_cles_fr(requete_utilisateur)
        print("Langue détectée ==> Français")
    else:
        mots_cles_format = extraire_mots_cles_en(requete_utilisateur)
        print("Langue détectée ==> Anglais")

    # Formatage des mots-clés pour une utilisation dans une URL de recherche
    search_query = format_mot(mots_cles_format)

    # Demander à l'utilisateur le nombre de pages à parcourir
    num_pages = int(input("Entrez le nombre de pages à parcourir : "))

    # Créer le nom du dossier de destination
    folder_name = f"{search_query.replace('+', '-')}-{num_pages}"

    # Créer le dossier s'il n'existe pas
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Initialiser le navigateur Chrome en mode sans tête
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    # Parcourir les pages demandées
    for page_num in range(1, num_pages+1):
        print(f"\nPage {page_num}")
        # Ouvrir la page de recherche de Semantic Scholar
        driver.get(f"https://www.semanticscholar.org/search?q={search_query}&sort=relevance&page={page_num}")

        # Attendre quelques secondes pour permettre au contenu de charger
        time.sleep(20)

        # Obtenir le contenu de la page
        html_content = driver.page_source

        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Trouver tous les liens vers les PDF
        pdf_links = soup.select('a[href$=".pdf"]')

        # Télécharger les PDF et les enregistrer dans le dossier "articles"
        for link in pdf_links:
            pdf_url = link["href"]
            print("Téléchargement du PDF:", pdf_url)
            try:
                # Ajouter un en-tête utilisateur pour simuler une demande de navigateur standard
                headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.1 Safari/537.36"}
                # Télécharger le PDF depuis le lien en incluant les en-têtes
                response = requests.get(pdf_url, headers=headers, stream=True)
                response.raise_for_status()  # Vérifier si la requête a réussi
                # Générer un nom de fichier unique pour chaque PDF
                file_name = f"article_{uuid.uuid4()}.pdf"  # Utiliser uuid.uuid4() pour générer un nom de fichier unique
                # Enregistrer le PDF dans le dossier de destination
                with open(os.path.join(folder_name, file_name), "wb") as f:
                    f.write(response.content)
                print(f"Enregistré sous: {folder_name}/{file_name}")
            except requests.exceptions.RequestException as e:
                print(f"Erreur lors du téléchargement du PDF {pdf_url}: {str(e)}")
            except Exception as ex:
                print(f"Une erreur inattendue s'est produite : {str(ex)}")

    # Fermer le navigateur
    driver.quit()

if __name__ == "__main__":
    main()


Veuillez saisir votre requête : I'm looking for articles on NLP
Langue détectée ==> Anglais
Entrez le nombre de pages à parcourir : 5

Page 1
Téléchargement du PDF: https://arxiv.org/pdf/1902.00751.pdf
Enregistré sous: NLP-5/article_b8cdcf69-a2d1-43a9-9a87-b0c4200d64f3.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/P19-1355.pdf
Enregistré sous: NLP-5/article_4c69a0fc-0fad-41fe-bfdd-68d19ac66ebc.pdf
Téléchargement du PDF: https://arxiv.org/pdf/2005.11401.pdf
Enregistré sous: NLP-5/article_51c6bfbc-e60d-474c-ae96-2275590f44ee.pdf
Téléchargement du PDF: https://www.aclanthology.org/2022.emnlp-main.340.pdf
Enregistré sous: NLP-5/article_10e0a06f-06a2-434e-b0d7-9e88bc9e4e47.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2020.acl-main.442.pdf
Enregistré sous: NLP-5/article_3700a370-835e-4d28-b6fd-c0bed26661c3.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2021.naacl-main.168.pdf
Enregistré sous: NLP-5/article_293f073b-0f29-45e5-b3d1-34da0b284251.pdf
Tél

In [15]:
import requests
from PyPDF2 import PdfReader

def download_pdf_and_extract_text(pdf_url):
    try:
        # Télécharger le fichier PDF depuis le lien
        response = requests.get(pdf_url)
        response.raise_for_status()  # Vérifier si la requête a réussi
        
        # Déterminer le nom du fichier
        file_name = pdf_url.split("/")[-1]

        # Enregistrer le PDF localement
        with open(file_name, "wb") as f:
            f.write(response.content)
        
        # Extraire le texte du PDF
        pdf_text = extract_text_from_pdf(file_name)
        return pdf_text
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement du PDF {pdf_url}: {str(e)}")
        return None
    except Exception as ex:
        print(f"Une erreur inattendue s'est produite : {str(ex)}")
        return None

def extract_text_from_pdf(pdf_file):
    try:
        # Ouvrir le fichier PDF en mode lecture binaire
        with open(pdf_file, "rb") as f:
            # Initialiser le lecteur PDF
            pdf_reader = PdfReader(f)
            
            # Extraire le texte de chaque page du PDF
            pdf_text = ""
            for page in pdf_reader.pages:
                pdf_text += page.extract_text()
            
            return pdf_text
    except Exception as e:
        print(f"Erreur lors de l'extraction du texte du PDF : {str(e)}")
        return None

# Lien PDF à télécharger
pdf_url = "https://scontent-cdg4-3.xx.fbcdn.net/v/t39.8562-6/246721374_422204999475172_9039387325224382577_n.pdf?_nc_cat=104&ccb=1-7&_nc_sid=e280be&_nc_ohc=f7UEYoVR9PAAb5Jmbyd&_nc_ht=scontent-cdg4-3.xx&oh=00_AfD1KZ-n5bA6jyE8wMo1rqiko8XoBL7d_vW4BgXCjybEcA&oe=662C70BA"

# Télécharger le PDF et extraire son texte
pdf_text = download_pdf_and_extract_text(pdf_url)
if pdf_text:
    print("Texte extrait du PDF :\n", pdf_text)
else:
    print("Impossible de télécharger ou extraire le texte du PDF.")


Texte extrait du PDF :
 Transformers4Rec: Bridging the Gap between NLP and
Sequential / Session-Based Recommendation
Gabriel de Souza Pereira
Moreira
gmoreira@nvidia.com
NVIDIA
São Paulo, BrazilSara Rabhi
srabhi@nvidia.com
NVIDIA
Ontario, CanadaJeong Min Lee∗
jeongmin@fb.com
Facebook AI
California, United States
Ronay Ak
ronaya@nvidia.com
NVIDIA
Florida, United StatesEven Oldridge
eoldridge@nvidia.com
NVIDIA
British Columbia, Canada
ABSTRACT
Much of the recent progress in sequential and session-based rec-
ommendation has been driven by improvements in model archi-
tecture and pretraining techniques originating in the field of Natu-
ral Language Processing. Transformer architectures in particular
have facilitated building higher-capacity models and provided data
augmentation and training techniques which demonstrably im-
prove the effectiveness of sequential recommendation. But with a
thousandfold more research going on in NLP, the application of
transformers for recommendation understa

# 4 Version 

In [23]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import spacy
from langdetect import detect
import uuid # Importer le module uuid pour la génération de noms de fichiers uniques
import random as rd

#user_agents = [
#    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.1 Safari/537.36",
#    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.1 Safari/537.36",
#    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.1 Safari/537.36",
    # Ajoutez d'autres User Agents au besoin
#]
user_agents = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
        ]

def extraire_mots_cles_fr(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en français.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("fr_core_news_sm")
    requete_utilisateur = requete.lower()
    doc = nlp(requete_utilisateur)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT","obj"]) or \
           (token.pos_ == "ADV" ) or \
           (token.pos_ == "PRON" and token.dep_ != "dep") or \
           (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or \
           (token.pos_ == "ADJ" ) or \
           (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]):
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def extraire_mots_cles_en(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en anglais.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(requete)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" ) or  (token.dep_ == "punct") or \
           (token.pos_ == "ADV") or (token.dep_ == "pobj") or \
           (token.pos_ == "PROPN" and token.dep_ != "dep") or \
           (token.pos_ == "ADJ" ) :
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def detect_language(text):
    """
    Fonction pour détecter la langue d'un texte donné.
    
    Arguments :
        - text : Le texte à analyser
        
    Returns :
        - 'français' si la langue détectée est le français, sinon 'anglais'
    """
    lang = detect(text)
    if lang == 'fr':
        return 'français'
    else:
        return 'anglais'

def format_mot(motcle):
    """
    Fonction pour formater un mot-clé pour une utilisation dans une URL de recherche.
    
    Arguments :
        - motcle : Le mot-clé à formater
        
    Returns :
        - format_motcle : Le mot-clé formaté pour une URL de recherche
    """
    if ' ' in motcle:
        format_motcle = motcle.replace(' ', '+')
    else:
        format_motcle = motcle
    return format_motcle

def download_pdf_and_save(pdf_url, folder_name):
    """
    Fonction pour télécharger un fichier PDF depuis un URL et l'enregistrer dans un dossier spécifié.
    
    Arguments :
        - pdf_url : L'URL du fichier PDF à télécharger
        - folder_name : Le nom du dossier dans lequel enregistrer le fichier PDF
        
    Returns :
        - file_path : Le chemin du fichier enregistré
    """
    try:
        headers = {"User-Agent": rd.choice(user_agents)}
        response = requests.get(pdf_url, headers=headers, stream=True)
        response.raise_for_status()
        file_name = f"article_{uuid.uuid4()}.pdf"
        file_path = os.path.join(folder_name, file_name)
        with open(file_path, "wb") as f:
            f.write(response.content)
        return file_path
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement du PDF {pdf_url}: {str(e)}")
        return None

def main():
    """
    Fonction principale du programme.
    """
    requete_utilisateur = input('Veuillez saisir votre requête : ')

    langue = detect_language(requete_utilisateur)

    if langue == 'français':
        mots_cles_format = extraire_mots_cles_fr(requete_utilisateur)
        print("Langue détectée ==> Français")
    else:
        mots_cles_format = extraire_mots_cles_en(requete_utilisateur)
        print("Langue détectée ==> Anglais")

    search_query = format_mot(mots_cles_format)

    num_pages = int(input("Entrez le nombre de pages à parcourir : "))
    folder_name = f"{search_query.replace('+', '-')}-{num_pages}"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    for page_num in range(1, num_pages+1):
        print(f"\nPage {page_num}")
        page_url = f"https://www.semanticscholar.org/search?q={search_query}&sort=relevance&page={page_num}"
        driver.get(page_url)
        time.sleep(5) 

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        pdf_links = soup.select('a[href$=".pdf"]')

        for link in pdf_links:
            pdf_url = link["href"]
            print("Téléchargement du PDF:", pdf_url)
            file_path = download_pdf_and_save(pdf_url, folder_name)
            if file_path:
                print(f"Enregistré sous: {file_path}")

    driver.quit()

if __name__ == "__main__":
    main()

Veuillez saisir votre requête : I'm looking for articles on NLP
Langue détectée ==> Anglais
Entrez le nombre de pages à parcourir : 6

Page 1
Téléchargement du PDF: https://arxiv.org/pdf/1902.00751.pdf
Enregistré sous: NLP-6/article_4dcd31f3-bf7c-44e9-b02c-bc12a4f73c7b.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/P19-1355.pdf
Enregistré sous: NLP-6/article_8fc51e08-dcca-45e4-9211-065abfc2b756.pdf
Téléchargement du PDF: https://arxiv.org/pdf/2005.11401.pdf
Enregistré sous: NLP-6/article_f8ecc57c-78f6-484a-8072-c072c14e9f9c.pdf
Téléchargement du PDF: https://www.aclanthology.org/2022.emnlp-main.340.pdf
Enregistré sous: NLP-6/article_5132eed7-f4e3-44a6-9871-6ff15e5e9ffa.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2021.naacl-main.168.pdf
Enregistré sous: NLP-6/article_a20b36a0-fbdb-4b90-b4c9-4e25b6a3ad9a.pdf
Téléchargement du PDF: https://www.aclweb.org/anthology/2020.acl-main.442.pdf
Enregistré sous: NLP-6/article_09f80e32-3b60-46ed-93b0-3c17461a1e91.pdf
Tél

Enregistré sous: NLP-6/article_a161ae6a-69a3-44e2-b32a-b08b63088168.pdf
Téléchargement du PDF: https://arxiv.org/pdf/2004.08900.pdf
Enregistré sous: NLP-6/article_a1e2f0d8-cd05-4df0-b37b-70de713c2c62.pdf
Téléchargement du PDF: https://www.aclanthology.org/2021.acl-long.431.pdf
Enregistré sous: NLP-6/article_1045f7e8-93a3-4cb3-8dcf-effb53488949.pdf
Téléchargement du PDF: https://www.aclanthology.org/2021.acl-long.431.pdf
Enregistré sous: NLP-6/article_231823ad-2159-408b-b475-936fe752dae2.pdf


In [17]:
import requests

def telecharger_pdf(url, nom_fichier):
    # Envoyer une requête GET pour récupérer le contenu du fichier
    response = requests.get(url)
    if response.status_code == 200:
        # Écrire le contenu de la réponse dans un fichier local
        with open(nom_fichier, 'wb') as f:
            f.write(response.content)
        print("Le fichier a été téléchargé et enregistré sous le nom:", nom_fichier)
    else:
        print("Impossible de télécharger le fichier. Statut de la requête:", response.status_code)

# Lien du PDF
url_pdf = "https://research.fb.com/wp-content/uploads/2021/09/Transformers4Rec-Bridging-the-Gap-between-NLP-and-Sequential-Session-Based-Recommendation-1.pdf"

# Nom du fichier local
nom_local = "YACINE.PDF"

# Appel de la fonction pour télécharger le PDF
telecharger_pdf(url_pdf, nom_local)


Le fichier a été téléchargé et enregistré sous le nom: YACINE.PDF


# <<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [25]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import spacy
from langdetect import detect
import uuid # Importer le module uuid pour la génération de noms de fichiers uniques
import random as rd

# Liste des User Agents
user_agents = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

def extraire_mots_cles_fr(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en français.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("fr_core_news_sm")
    requete_utilisateur = requete.lower()
    doc = nlp(requete_utilisateur)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT","obj"]) or \
           (token.pos_ == "ADV" ) or \
           (token.pos_ == "PRON" and token.dep_ != "dep") or \
           (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or \
           (token.pos_ == "ADJ" ) or \
           (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]):
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def extraire_mots_cles_en(requete):
    """
    Fonction pour extraire les mots-clés à partir de la requête de l'utilisateur en anglais.
    
    Arguments :
        - requete : La requête de l'utilisateur
        
    Returns :
        - mots_cles_format : Les mots-clés formatés sous forme de chaîne de caractères
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(requete)
    mots_cles = []
    for token in doc:
        if (token.pos_ == "NOUN" ) or  (token.dep_ == "punct") or \
           (token.pos_ == "ADV") or (token.dep_ == "pobj") or \
           (token.pos_ == "PROPN" and token.dep_ != "dep") or \
           (token.pos_ == "ADJ" ) :
            if not token.is_stop and not token.is_punct and token.text.lower() not in ['article', 'articles']:
                mots_cles.append(token.text)
    mots_cles_format = ' '.join(mots_cles)
    return mots_cles_format

def detect_language(text):
    """
    Fonction pour détecter la langue d'un texte donné.
    
    Arguments :
        - text : Le texte à analyser
        
    Returns :
        - 'français' si la langue détectée est le français, sinon 'anglais'
    """
    lang = detect(text)
    if lang == 'fr':
        return 'français'
    else:
        return 'anglais'

def format_mot(motcle):
    """
    Fonction pour formater un mot-clé pour une utilisation dans une URL de recherche.
    
    Arguments :
        - motcle : Le mot-clé à formater
        
    Returns :
        - format_motcle : Le mot-clé formaté pour une URL de recherche
    """
    if ' ' in motcle:
        format_motcle = motcle.replace(' ', '+')
    else:
        format_motcle = motcle
    return format_motcle

def download_pdf_and_save(pdf_url, folder_name):
    """
    Fonction pour télécharger un fichier PDF depuis un URL et l'enregistrer dans un dossier spécifié.
    
    Arguments :
        - pdf_url : L'URL du fichier PDF à télécharger
        - folder_name : Le nom du dossier dans lequel enregistrer le fichier PDF
        
    Returns :
        - file_path : Le chemin du fichier enregistré
    """
    try:
        headers = {"User-Agent": rd.choice(user_agents)}
        response = requests.get(pdf_url, headers=headers, stream=True)
        response.raise_for_status()
        file_name = f"article_{uuid.uuid4()}.pdf"
        file_path = os.path.join(folder_name, file_name)
        with open(file_path, "wb") as f:
            f.write(response.content)
        return file_path
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement du PDF {pdf_url}: {str(e)}")
        return None

def main():
    """
    Fonction principale du programme.
    """
    requete_utilisateur = input('Veuillez saisir votre requête : ')

    langue = detect_language(requete_utilisateur)

    if langue == 'français':
        mots_cles_format = extraire_mots_cles_fr(requete_utilisateur)
        print("Langue détectée ==> Français")
    else:
        mots_cles_format = extraire_mots_cles_en(requete_utilisateur)
        print("Langue détectée ==> Anglais")

    search_query = format_mot(mots_cles_format)

    num_pages = int(input("Entrez le nombre de pages à parcourir : "))
    folder_name = f"{search_query.replace('+', '-')}-{num_pages}"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    num_articles = 0  # Compteur d'articles téléchargés avec succès

    for page_num in range(1, num_pages+1):
        print(f"Page {page_num}")
        page_url = f"https://www.semanticscholar.org/search?q={search_query}&sort=relevance&page={page_num}"
        driver.get(page_url)
        time.sleep(5) 

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        pdf_links = soup.select('a[href$=".pdf"]')

        for link in pdf_links:
            pdf_url = link["href"]
            #print("Téléchargement du PDF:", pdf_url)
            file_path = download_pdf_and_save(pdf_url, folder_name)
            if file_path:
                num_articles += 1
                #print(f"Enregistré sous: {file_path}")

    driver.quit()

    print(f"Nombre total d'articles téléchargés avec succès : {num_articles}")

if __name__ == "__main__":
    main()


Veuillez saisir votre requête : I'm looking for articles on NLP
Langue détectée ==> Anglais
Entrez le nombre de pages à parcourir : 5
Page 1
Page 2
Page 3
Page 4
Page 5
Nombre total d'articles téléchargés avec succès : 49
